# MFW 6 + flopy + matplotlib

Autor: Luis M. de la Cruz


## Paso 1. Importamos las bibliotecas
Importamos todas las bibliotecas `os` (para acceso al sistema operativo), `numpy` para manejo de arreglos, `matplotlib` para graficación y `flopy`.

In [ ]:
# Cuando trabajas en MACTI, debes forzosamente instalar flopy como sigue 
# (descomenta la línea siguiente):
!pip install flopy

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy

## Paso 2. Definimos la ruta de MODFLOW
Definimos la ruta donde se encuentra el ejecutable de MODFLOW 6 y el nombre de la simulación.

In [ ]:
ws = os.getcwd() # Ruta de donde estamos actualmente

# A continuación modifica la línea correspondiente, de acuerdo con tu sistema
# operativo y tu ambiente. La variable mf6_exe define la ruta del ejecutable 
# de MODFLOW6

# WINDOWS
#mf6_exe = 'C:\\Users\\luiggi\\Downloads\\mf6.4.2\\mf6.4.2_win64\\bin\\mf6.exe'

# MAC 
#mf6_exe = '/Users/luiggi/GitSites/mf6.4.4_mac/bin/mf6'

# LINUX (MACTI) 
mf6_exe = '../../mf6/bin/mf6'

# Definimos el nombre de la simulación
name = 'quick_start'

print(' Ruta hacia el ejecutable de MODFLOW 6 : {} '.format(mf6_exe))
print(' Nombre de esta simulación: {}'.format(name))

## Paso 3. Objeto para la simulación
Creamos un objeto de tipo `MFSimulation`

In [ ]:
sim  = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws, exe_name=mf6_exe)

In [ ]:
print(type(sim),'\n')
print(sim)

## Paso 4. Objeto para el tiempo
Creamos un objeto de tipo `ModflowTDis` para la discretización en el tiempo

In [ ]:
tdis = flopy.mf6.ModflowTdis(sim)

In [ ]:
print(type(tdis),'\n')
print(tdis)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 5. Objeto para la solución.

Creamos un objeto de tipo `ModflowIms` para la solución iterativa

In [ ]:
ims  = flopy.mf6.ModflowIms(sim)

In [ ]:
print(type(ims), '\n')
print(ims)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 6. Objeto para el modelo de flujo
Creamos un modelo de flujo usando `ModflowGwf`

In [ ]:
gwf  = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)

In [ ]:
print(type(gwf), '\n')
print(gwf)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 7. Objeto para la malla

Creamos un objeto para la discretización espacial
En Modflow 6 existen tres maneras:
- DIS (ModflowGwfdis) - Structured discretization
- DISV (ModflowGwfdisv) - Discretization with vertices
- DISU (ModflowGwfdisu) - Unstructured discretization

In [ ]:
nrow = 10
ncol = 10
dis  = flopy.mf6.ModflowGwfdis(gwf, nrow=nrow, ncol=ncol)

In [ ]:
print(type(dis), '\n')
print(dis)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 8. Condiciones iniciales

Creamos un objeto de tipo `ModflowGwfic` para las condiciones iniciales.

In [ ]:
ic   = flopy.mf6.ModflowGwfic(gwf)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 9. Propiedades de los nodos

Creamos un objeto de tipo `ModflowGwfnpf` para definir propiedades en los nodos y otro de tipo `ModflowGwfchd` para definir la carga constante

In [ ]:
npf  = flopy.mf6.ModflowGwfnpf(gwf, save_specific_discharge=True)

In [ ]:
print(type(npf),'\n')
print(npf)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

In [ ]:
xc = nrow-1
yc = ncol-1
chd  = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=[[(0, 0, 0), 10.],
                                                        [(0, int(xc/2), int(yc/2)), 2.5],
                                                       [(0, xc, yc), 0.]])

In [ ]:
print(xc, yc)

In [ ]:
print(type(chd),'\n')
print(chd)

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 10. Objeto para la salida.
Creamos un objeto de tipo `ModflowGwfoc` para la salida de la simulación

In [ ]:
budget_file = name + '.bud'
head_file = name + '.hds'

oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord=budget_file,
                            head_filerecord=head_file,
                            saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')])


In [ ]:
print(type(oc),'\n')
print(oc)

## Paso 11. Archivos de entrada.
Escribimos los datos de entrada para la simulación

In [ ]:
# Escribo los datos definidos hasta ahora en los archivos correspondientes
sim.write_simulation()

## Paso 12. Ejecución
Ejecutamos la simulación

In [ ]:
sim.run_simulation()

## Paso 13. Recuperamos la salida.
Realizamos la visualización

In [ ]:
# Obtenemos los resultados de la carga hidráulica
head = flopy.utils.HeadFile(os.path.join(ws, head_file)).get_data()

# Obtenemos los resultados del BUDGET
bud  = flopy.utils.CellBudgetFile(os.path.join(ws, budget_file),
                                  precision='double')

# Obtenemos las velocidades
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)

In [ ]:
# Verificamos el tipo y dimensiones de los arreglos donde
# están almacenados la carga hidráulica, el BUDGET, y la velocidad.
print('Head : ', type(head), head.shape)
print('Budget : ', type(bud), bud.shape)
print('qx : ', type(qx), qx.shape)
print('qy : ', type(qy), qy.shape)
print('qz : ', type(qz), qz.shape)

## Paso 14. Visualización.

Podemos usar las herramientas de visualización de flopy y combinarlas con herramientas de Matplotlib.

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
ax[0,0].set_aspect('equal')
ax[0,1].set_aspect('equal')
ax[1,0].set_aspect('equal')
ax[1,1].set_aspect('equal')

# Dibujamos la malla (con flopy)
pmv0 = flopy.plot.PlotMapView(gwf, ax=ax[0,0])
pmv0.plot_grid(colors='dimgray', lw=0.5)

# Dibujamos la carga hidráulica (con flopy)
pmv1 = flopy.plot.PlotMapView(gwf, ax=ax[0,1])
pmv1.plot_array(head)
pmv1.contour_array(head, levels=10, colors='white', linewidths=1.)

# Dibujamos la carga velocidad (con flopy)
pmv2 = flopy.plot.PlotMapView(gwf, ax=ax[1,0])
pmv2.plot_vector(qx, qy, normalize=False, color="k")

# Matplotlib para dibujar streamlines
Lx = nrow
Ly = ncol
delc = Lx / ncol
delr = Ly / nrow
# Malla donde se dibuja la velocidad
x = np.linspace(0, Lx, ncol)
y = np.linspace(0, Ly, nrow)
xg, yg = np.meshgrid(x,y)
ax[1,1].streamplot(xg,yg,qx[0],qy[0], color='k')
ax[1,1].set_xlim((0,Lx))
ax[1,1].set_ylim(0,Ly)